# MACE experiment notebook 

This experiment notebook tries to understand how to speed up varius part of [MACE](../Notes/General%20Concepts/MACE.md).

It relies on the different tutorial notebooks proposed by the [MACE repositery](https://github.com/ACEsuit/mace).

## Precision Theory

✅ Summary: FP32 vs FP16

| Property          | FP32 (float32)       | FP16 (float16)                                          |
| ----------------- | -------------------- | ------------------------------------------------------- |
| Total bits        | 32                   | 16                                                      |
| Mantissa bits     | 23                   | 10                                                      |
| Decimal precision | \~7 digits           | \~3 digits                                              |
| Value range       | \~±$10^{38}$         | \~±$10^5$                                               |
| Memory usage      | 4 bytes / value      | 2 bytes / value                                         |
| Performance       | Slower, more precise | Faster, less precise                                    |
| Typical use       | Default for training | Efficient inference / low-precision training (with AMP) |



import torch

# Define a float32 tensor with high-precision value
vector_fp32 = torch.tensor([1.2345678], dtype=torch.float32)

# Convert to float16 (half precision)
vector_fp16 = vector_fp32.half()

# Print both values with full decimal precision
print(f"FP32 value  : {vector_fp32.item():.10f}")
print(f"FP16 value  : {vector_fp16.item():.10f}")

# Absolute error
error = torch.abs(vector_fp16.float() - vector_fp32)
print(f"Absolute error: {error.item():.10f}")
```

## T03-MACE-Theory.ipynb

### Install dependencies

In [16]:
from mace import data, modules, tools
import numpy as np
import torch
import torch.nn.functional
from e3nn import o3
from matplotlib import pyplot as plt
import ase.io
%matplotlib inline
from ase.visualize import view
from scipy.spatial.transform import Rotation

from mace.tools import torch_geometric
# torch.set_default_dtype(torch.float64)
import warnings
warnings.filterwarnings("ignore")

### Speeding up Rotational Embedding

Speeding up the computation of spherical harmonics $Y_l^m(\hat{\mathbf{r}})$, and how do I verify whether lowering its precision has any significant impact?

In [6]:
# a function for Ylms where we evaluate for l=0,1,2.
spherical_harmonics = o3.SphericalHarmonics([0,1,2], True)

print(spherical_harmonics)

# evaulate spherical harmonics on a vector
vector = torch.tensor([1.0, 0.2, 0.75])
print(spherical_harmonics(vector))

SphericalHarmonics()
tensor([ 0.2821,  0.3860,  0.0772,  0.2895,  0.5113,  0.1364, -0.2918,  0.1023,
        -0.1491])


In [24]:

torch.set_default_dtype(torch.float32)
vector_fp32 = torch.tensor([1.2345678])
print(f"vector_fp32: {vector_fp32} - we can see that the tensor is rounding from 1.2345678 to 1.2346")
print(vector_fp32.dtype)
print(f"{vector_fp32.item():.10f}")  # → 1.2345678806
vector_fp16 = vector_fp32.half()
print(f"{vector_fp16.item():.10f}")  # → 1.2343750000
print(vector_fp16.dtype)
print(f"vector_fp16: {vector_fp16} - we can see that the tensor is rounding from 1.2345678 to 1.2344")  # Output: tensor([1.234375], dtype=torch.float16)


vector_fp32: tensor([1.2346]) - we can see that the tensor is rounding from 1.2345678 to 1.2346
torch.float32
1.2345677614
1.2343750000
torch.float16
vector_fp16: tensor([1.2344], dtype=torch.float16) - we can see that the tensor is rounding from 1.2345678 to 1.2344
